## Modified from Unsloth Tutorials

### Installation

In [1]:
import sys
print(sys.version)

3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]


In [2]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5.

In [3]:
import json
def transform_json(input_path, output_path):
    # Load the original JSON file
    with open(input_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    transformed = []

    for item in data.get("examples", []):
        # Ensure 'input' and 'response' keys exist
        input_text = item.get("input")
        response_text = item.get("response")

        if input_text is None or response_text is None:
            continue  # Skip invalid entries

        transformed.append({
            'conversations': [
                {'from': 'human', 'value': input_text},
                {'from': 'gpt', 'value': response_text}
            ]
        })

    # Save the transformed JSON to a new file
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(transformed, file, indent=4, ensure_ascii=False)

transform_json('read_aggregated.json', 'read_aggregated_parsed.json')
transform_json('write_aggregated.json', 'write_aggregated_parsed.json')

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [5]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    # model_name = "unsloth/gemma-3-4b-it",
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length = 1024,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.3: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [6]:
model_ask, tokenizer_ask = model, tokenizer

We now add LoRA adapters so we only need to update a small amount of parameters!

In [7]:
model_ask = FastModel.get_peft_model(
    model_ask,
    finetune_vision_layers     = False,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 8,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [11]:
from unsloth.chat_templates import get_chat_template
tokenizer_ask = get_chat_template(
    tokenizer_ask,
    chat_template = "gemma-3",
)

In [12]:
from datasets import load_dataset

In [17]:
ask_dataset = load_dataset("json", data_files="read_aggregated_parsed.json", split = "train")

In [18]:
ask_dataset

Dataset({
    features: ['conversations'],
    num_rows: 1068
})

In [19]:
ask_dataset[0]

{'conversations': [{'from': 'human', 'value': "What's the weather in London?"},
  {'from': 'gpt', 'value': '<memory_ask>Weather in London</memory_ask>'}]}

In [20]:
from unsloth.chat_templates import standardize_data_formats
ask_dataset = standardize_data_formats(ask_dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/1068 [00:00<?, ? examples/s]

Let's see how row 100 looks like!

In [21]:
ask_dataset[2]

{'conversations': [{'content': 'Tell me a joke.', 'role': 'user'},
  {'content': '<memory_ask>Retrieve a joke</memory_ask>',
   'role': 'assistant'}]}

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`

In [22]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
ask_dataset = ask_dataset.map(apply_chat_template, batched = True)

Map:   0%|          | 0/1068 [00:00<?, ? examples/s]

Let's see how the chat template did! Notice `Gemma-3` default adds a `<bos>`!

In [23]:
ask_dataset[100]["text"]

"<bos><start_of_turn>user\nWhat is 'Dimensionality Reduction' in machine learning?<end_of_turn>\n<start_of_turn>model\n<memory_ask>Definition of Dimensionality Reduction in machine learning</memory_ask><memory_ask>Purpose of Dimensionality Reduction (e.g., reduce complexity, improve performance)</memory_ask><memory_ask>Common techniques for Dimensionality Reduction (e.g., PCA, t-SNE)</memory_ask><end_of_turn>\n"

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [24]:
from trl import SFTTrainer, SFTConfig
trainer_ask = SFTTrainer(
    model = model_ask,
    tokenizer = tokenizer_ask,
    train_dataset = ask_dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5,
        # max_steps = 30,
        # learning_rate = 2e-4,
        learning_rate = 2e-5,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1068 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [25]:
from unsloth.chat_templates import train_on_responses_only
trainer_ask = train_on_responses_only(
    trainer_ask,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/1068 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again:

In [26]:
tokenizer_ask.decode(trainer_ask.train_dataset[100]["input_ids"])

"<bos><bos><start_of_turn>user\nWhat is 'Dimensionality Reduction' in machine learning?<end_of_turn>\n<start_of_turn>model\n<memory_ask>Definition of Dimensionality Reduction in machine learning</memory_ask><memory_ask>Purpose of Dimensionality Reduction (e.g., reduce complexity, improve performance)</memory_ask><memory_ask>Common techniques for Dimensionality Reduction (e.g., PCA, t-SNE)</memory_ask><end_of_turn>\n"

Now let's print the masked out example - you should see only the answer is present:

In [30]:
tokenizer_ask.decode([tokenizer_ask.pad_token_id if x == -100 else x for x in trainer_ask.train_dataset[100]["labels"]]).replace(tokenizer_ask.pad_token, " ")

'                     <memory_ask>Definition of Dimensionality Reduction in machine learning</memory_ask><memory_ask>Purpose of Dimensionality Reduction (e.g., reduce complexity, improve performance)</memory_ask><memory_ask>Common techniques for Dimensionality Reduction (e.g., PCA, t-SNE)</memory_ask><end_of_turn>\n'

In [31]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.57 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [32]:
trainer_stats = trainer_ask.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,068 | Num Epochs = 5 | Total steps = 665
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.666500
2,3.753600
3,2.615600
4,2.810300
5,2.233100
6,2.423900
7,2.797500
8,2.640500
9,3.690200
10,2.587100


In [33]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2856.549 seconds used for training.
47.61 minutes used for training.
Peak reserved memory = 5.57 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 37.786 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [37]:
from unsloth.chat_templates import get_chat_template
tokenizer_ask = get_chat_template(
    tokenizer_ask,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Who is John?",
    }]
}]
text = tokenizer_ask.apply_chat_template(
    messages,
    add_generation_prompt = True,
)
outputs = model_ask.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer_ask.batch_decode(outputs)

['<bos><start_of_turn>user\nWho is John?<end_of_turn>\n<start_of_turn>model\n<memory_ask>Information about John (character from Lord of the Rings)</memory_ask><memory_ask>Information about John (computer personality, chatbot)</memory_ask><end_of_turn>']

In [38]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<memory_ask>Explanation of why the sky is blue</memory_ask><memory_ask>Role of Rayleigh scattering in blue sky phenomenon</memory_ask><end_of_turn>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# model.save_pretrained("gemma-3")  # Local saving
# tokenizer.save_pretrained("gemma-3")
model_ask.push_to_hub("Pulkith/gemma-3-ask-finetune", token = "HF_KEY") # Online saving
tokenizer_ask.push_to_hub("Pulkith/gemma-3-ask-finetune", token = "HF_KEY") # Online saving

No files have been modified since last commit. Skipping to prevent empty commit.


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down what Gemma-3 is. It's a fascinating development in the world of AI, and here's a comprehensive overview:

**1. What it is:**

* **A Family of Open-Weight Language Models:** Gemma-3 isn't just *one* model


### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:

model_ask.push_to_hub_merged(
    "Pulkith/gemma-3-ask-float16", tokenizer_ask,
    token = "HF_KEY"
)

  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!